# Creating and Visualizing DEMs from LIDAR points

_**Caitlin Haedrich and Pratikshya Regmi**, North Carolina State University_

In this notebook we will:
* Create high-quality DEMs from LiDAR point clouds and compute topographic parameters
* Create webmap visualizations


***

## 1. Import Python Packages and Start GRASS GIS Session

Import Python standard library and IPython packages we need. Start GRASS session in Nags Head project.

In [ ]:
import subprocess
import sys
from pathlib import Path

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True, shell=False).strip()
)

import grass.script as gs
import grass.jupyter as gj

gj.init("./nags_head/PERMANENT");

In [ ]:
!g.region -p region=jockeys_ridge

**Try it yourself!**

_Did you modify the computational region at the end of the previous notebook? If so, note if those changes are reflected in this notebook. Use `g.region` to switch back to the `jockeys_ridge` saved region._

<details>
    <summary>👉 <b>click for solution</b></summary>
    
The computational region is saved in the mapset so moving to a new notebook or starting a new session will not affect it.
```
!g.region -p region=jockeys_ridge
```
</details>

***

## 2. Create a DEM 

In [ ]:
lidar_files = sorted(Path('/data/grass-workshop').glob('*.las'))
lidar_files

### _Optional: What resolution should we use?_

To determine an appropriate resolution for our DEM, we use `r.in.pdal` to create a raster with the point count in each cell and vary the resolution. If we have at least 1 point per cell, we can use a binning approach. If we'd like to have a higher resolution than that, we should use a binning approach.

In [ ]:
res_options = [1,2,5,10]

for res in res_options:
    # Set the resolution
    gs.run_command("g.region", res=res)
    # Import the lidar data as a raster with count method which returns the points per cell
    gs.run_command("r.in.pdal", input="/data/grass-workshop/JR_2014.las", output="JR_2014_count", method="n", flags="w")
    # Compute univariate raster stats
    stats = gs.parse_command("r.univar", flags='ge', map="JR_2014_count")
    print(f"{res}m     Mean points per cell: {stats["mean"]}")

### Mask low density areas

Interpolating or creating a binned a DEM from LiDAR points is only meaningful where there is adequate point coverage. So, our first step will be to mask areas that have low point densities before we create our DEM. This also will make our code run faster!

In [ ]:
!g.region region="jockeys_ridge" res=2

In [ ]:
gs.run_command("r.in.pdal", input="/data/grass-workshop/JR_2014.las", output="JR_2014_count", method="n", flags="w")
gs.mapcalc(exp="JR_2014_mask=if(JR_2014_count == 0, null(), 1 )")

In [ ]:
mask = gj.Map(use_region=True)
mask.d_rgb(red="naip_2014.1", green="naip_2014.2", blue="naip_2014.3")
mask.d_rast(map="JR_2014_mask")
mask.show()

In [ ]:
!r.mask raster=JR_2014_mask

### Create DEM with Binning

In [ ]:
gs.run_command("r.in.pdal", input="/data/grass-workshop/JR_2014.las", output="JR_2014_raw", method="mean", flags="w")

In [ ]:
!r.mask -r

In [ ]:
gs.run_command("r.fill.stats", input="JR_2014_raw", output="JR_2014", distance=5, mode="wmean", power=2.0, cells=8)

You can also compute DEMs with a higher resolution than the point sampling distance using splines with either the [v.surf.rst](https://grass.osgeo.org/grass83/manuals/v.surf.rst.html) or [v.surf.bspline](https://grass.osgeo.org/grass83/manuals/v.surf.bspline.html) tools.

See Section 2.3.3 of [Hardin et al (2014)](https://link.springer.com/chapter/10.1007/978-1-4939-1835-5_2#Sec3) for more.

### Visualize DEM

_A static map made with_ `gj.Map`.

In [ ]:
elev = gj.Map()
elev.d_rast(map="JR_2014")
elev.show()

_A leaflet map made with_ `gj.InteractiveMap()`

In [ ]:
fig = gj.InteractiveMap(width=800, tiles="OpenStreetMap")
fig.add_raster("JR_2014")
fig.add_layer_control()
fig.show()

### Create DEM with Splining

If we'd like to create a DEM with less than 1 point per cell, we'll have to use an interpolation approach. Here's an example showing how to do a regularized spline with tension in GRASS. We will:
1. Set the computational region
2. Import the point cloud as vector points
3. Use a regularized spline with tension (rst) to interpolate the surface from the points

Splining is computational intensive so we'll shrink our region by a little bit to help. Because we'll be interpolating from points, we can create a higher resolution raster. We'll set the resolution to 2 meters here.

In [ ]:
!g.region -p res=1 grow=-200

Import the bare earth point cloud as a vector using PDAL. The "w" flag will reproject the points if necessary. The "r" will limit the import to only the current computational region and the "c" flag 

In [ ]:
gs.run_command("v.in.pdal", input="/data/grass-workshop/JR_2014.las", output="JR_2014", flags="wrc") 

Now, we can interpolate using the RST algorithm. Visit [the manual page](https://grass.osgeo.org/grass83/manuals/v.surf.rst.html) for more info.

In [ ]:
# Interpolate using RST
gs.run_command("v.surf.rst", input="JR_2014", elev="JR_2014_spline", tension=40, npmin=100, segmax=15, dmin=2)

In [ ]:
gs.run_command("r.colors", map="JR_2014", color="elevation")

In [ ]:
fig = gj.InteractiveMap(width=800, tiles="OpenStreetMap")
fig.add_raster("JR_2014_spline")
fig.add_raster("JR_2014")
fig.add_layer_control()
fig.show()

**Try it yourself!**

`v.surf.rst` _is a powerful tool for interpolating LiDAR surfaces [(Mitasova et al, 2005)](https://ieeexplore.ieee.org/document/1522204). Look at the documentation for v.surf.rst and adjust the parameters to see how they affect the resulting DEM. How does tension change the result? What happens if you change the resolution of the computational region?_

<details>
    <summary>👉 <b>click to see an example</b></summary>

```
! v.surf.rst --help
``` 
    
```python
gs.run_command("v.surf.rst", input="JR_2014", elev="JR_2014_spline", tension=10, smooth=0)
```
</details>

## 3. Bulk DEM Creation with Python

In [ ]:
lidar_files 

Some of our files have very low resolution and will work better with splines. Some others have greater point density and can be binned. We'll make a function for each type of import.

In [ ]:
def binning_import(file, res):
    gs.run_command("g.region", region="jockeys_ridge", res=res)
    
    # Create density mask
    output_name = file.stem
    gs.run_command("r.in.pdal", input=file, output=output_name+'_count', method="n", flags="w")
    gs.mapcalc(exp=f"{output_name}_mask=if({output_name}_count == 0, null(), 1 )")
    
    # Set Mask
    gs.run_command("r.mask", raster=f"{output_name}_mask")

    # Binning
    gs.run_command("r.in.pdal", input=file, output=f"{output_name}_raw", method="mean", flags="w") #create binned raster
    gs.run_command("r.mask", flags="r") # remove mask
    gs.run_command("r.fill.stats", input=f"{output_name}_raw", output=output_name, distance=5, mode="wmean", power=2.0, cells=8)
    print(f"imported {file.stem}")

In [ ]:
def rst_import(file, res):
    
    # Create density mask
    output_name = file.stem
    gs.run_command("g.region", region="jockeys_ridge", res=20) # Let's create a mask so we don't interpolate over open ocean or anything
    gs.run_command("r.in.pdal", input=file, output=output_name+'_count', method="n", flags="w")
    gs.mapcalc(exp=f"{output_name}_mask=if({output_name}_count == 0, null(), 1 )")

    # Reset resolution to desired res, import and spline
    gs.run_command("g.region", res=res)
    gs.run_command("v.in.pdal", input=file, output=output_name, flags="wrc")
    gs.run_command(
        "v.surf.rst",
        input=output_name,
        elev=f"{output_name}",
        tension=40,
        npmin=50,
        segmax=30,
        mask=f"{output_name}_mask"
    )
    print(f"imported {file.stem}")

Then, we import all the files!

In [ ]:
binned = lidar_files[1:4]+lidar_files[5:]
binned

In [ ]:
# 4 meter resolution binned
for file in binned:
    binning_import(file, 4)

### Splined clouds (1996, 1999_0918)
rst_import(lidar_files[0], 4)
rst_import(lidar_files[4], 4)

**Try it yourself!**

_Visualize some of the DEMs we created. Use r.info to learn more about them._